# Ensemble Learning

## Initial Imports

In [153]:
import warnings
warnings.filterwarnings('ignore')

In [154]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [155]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [156]:
# Load the data
file_path = Path(r'Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [157]:
# Create our features
X = df.drop(columns=["loan_status"])
X = pd.get_dummies(X)
# Create our target
from sklearn.preprocessing import StandardScaler, LabelEncoder
y = df["loan_status"]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [158]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [159]:
# Convert y to a pandas Series
y_series = pd.Series(y)

# Check the balance of our target values
y_series.value_counts()

1    68470
0      347
dtype: int64

In [160]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [161]:
# Create the StandardScaler instance

scaler = StandardScaler()

In [162]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)


StandardScaler()

In [163]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [164]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier
clf = BalancedRandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=4,
    max_features='sqrt',
    bootstrap=True,
    random_state=42
)

clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)



In [165]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7395520354170544

In [166]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   47,    41],
       [  752, 12924]], dtype=int64)

In [167]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.06      0.53      0.95      0.11      0.71      0.48        88
          1       1.00      0.95      0.53      0.97      0.71      0.53     13676

avg / total       0.99      0.94      0.54      0.96      0.71      0.53     13764



In [168]:
# List the features sorted in descending order by feature importance
importances = clf.feature_importances_

# Step 1: Extract feature names
feature_names = X.columns.tolist()

# Step 2: Sort the features using your code
sorted_indices = np.argsort(importances)[::-1]

# Step 3: Print the sorted features
for idx in sorted_indices:
    print(feature_names[idx], ":", importances[idx])

total_rec_prncp : 0.13675930515718682
total_pymnt_inv : 0.10996531901448486
total_pymnt : 0.1067763551936274
total_rec_int : 0.09852381638091562
last_pymnt_amnt : 0.087752684735277
int_rate : 0.029549510608601354
issue_d_Mar-2019 : 0.022322346137244892
issue_d_Jan-2019 : 0.018953749255128095
out_prncp_inv : 0.013339605040526782
installment : 0.013265865311297073
dti : 0.012668280796680524
total_rec_late_fee : 0.012535184328918184
max_bal_bc : 0.012397298277213777
mo_sin_old_rev_tl_op : 0.01171111217267391
out_prncp : 0.011607958211424744
total_bc_limit : 0.01147246045808123
il_util : 0.011329236725233536
total_rev_hi_lim : 0.010287135465714244
bc_util : 0.010136435202286166
loan_amnt : 0.010087861212001928
revol_bal : 0.009969303485706772
annual_inc : 0.00934818589070829
tot_cur_bal : 0.008527734437823342
tot_hi_cred_lim : 0.008046785813305764
mo_sin_old_il_acct : 0.007309496386687913
inq_fi : 0.007302200409810716
acc_open_past_24mths : 0.0072641569918834945
mths_since_recent_inq : 0.0

### Easy Ensemble Classifier

In [169]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(
    n_estimators=100,
    base_estimator=None,
    warm_start=False,
    sampling_strategy='auto',
    replacement=False,
    n_jobs=-1,
    random_state=42
)

eec.fit(X_train_scaled, y_train)

y_pred = eec.predict(X_test_scaled)


In [170]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9173336169533888

In [171]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   78,    10],
       [  707, 12969]], dtype=int64)

In [172]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.10      0.89      0.95      0.18      0.92      0.84        88
          1       1.00      0.95      0.89      0.97      0.92      0.85     13676

avg / total       0.99      0.95      0.89      0.97      0.92      0.85     13764



### Final Questions

1. Which model had the best balanced accuracy score?

    the Ensemble Classifier

2. Which model had the best recall score?

    the Ensemble Classifier

3. Which model had the best geometric mean score?

    the Ensemble Classifier

4. What are the top three features?

    total_rec_prncp : 0.13675930515718682
    total_pymnt_inv : 0.10996531901448486
    total_pymnt : 0.1067763551936274